## Importando Bibliotecas

In [ ]:
import pandas as pd
import numpy as np

## Importando arquivos

In [ ]:
df = pd.read_csv('../input/train.csv', parse_dates=[0])
test = pd.read_csv('../input/test.csv', parse_dates=[0])

## Mudando o Nome da Coluna count

In [ ]:
df.rename(columns={'count':'rentals'}, inplace=True)

## Pegando o log das colunas que iremos prever

In [ ]:
for col in ['rentals', 'registered', 'casual']:
    df[col] = np.log(df[col] + 1)

## Pré-processamento dos Dados

Juntando os `DataFrames`

In [ ]:
df = df.append(test)

Tratamento de datas

In [ ]:
df['year'] = df['datetime'].dt.year
df['month'] = df['datetime'].dt.month
df['day'] = df['datetime'].dt.day
df['dayofweek'] = df['datetime'].dt.dayofweek
df['hour'] = df['datetime'].dt.hour

Criando a coluna `rolling_temp`

In [ ]:
df.set_index('datetime', inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
df['rolling_temp'] = df['temp'].rolling(4, min_periods=1).mean()

In [ ]:
df.reset_index(inplace=True)

Separando os `DataFrames`:

In [ ]:
test = df[df['rentals'].isnull()]

In [ ]:
df = df[~df['rentals'].isnull()]

## Selecionando as Colunas que Iremos Executar o Modelo

In [ ]:
removed_cols = ['rentals', 'casual', 'registered', 'datetime']

In [ ]:
feats = [c for c in df.columns if c not in removed_cols]

## Criando um Set de Validação Melhor

In [ ]:
train, valid = df[df['day'] <= 15], df[df['day'] > 15]

In [ ]:
train.shape, valid.shape

In [ ]:
train['day'].value_counts().sort_index()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=200,
                          min_samples_split=4, max_features=0.9,
                          max_depth=17, oob_score=True)

In [ ]:
rf.fit(train[feats], train['rentals'])

In [ ]:
preds = rf.predict(valid[feats])

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(valid['rentals'], preds)**(1/2)

In [ ]:
# Removendo a coluna day
feats = [c for c in feats if c not in ['day']]

In [ ]:
# Testando de novo sem a coluna day
rf = RandomForestRegressor(random_state=42, n_jobs=-1, n_estimators=200,
                           min_samples_leaf=10, oob_score=True, max_features=0.9)

In [ ]:
rf.fit(train[feats], train['rentals'])

In [ ]:
preds = rf.predict(valid[feats])

In [ ]:
mean_squared_error(valid['rentals'], preds)**(1/2)

In [ ]:
train_preds = rf.predict(train[feats])

In [ ]:
mean_squared_error(train['rentals'], train_preds)**(1/2)